# Burgers Equation Identification Using ANN-Adam Optimizer
Equation:   $u_{t} + \lambda_1 uu_{x}-\lambda_2 u_{xx} = 0$  

In [5]:
#author : $um@nth
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
import scipy.io
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [6]:
N_u = 2500
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat') 
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  
u_star = Exact.flatten()[:,None]                                                 
lb = X_star.min(0)                                                              
ub = X_star.max(0)
np.random.seed(107)
idx = np.random.choice(X_star.shape[0], N_u, replace=False)

X_train = X_star[idx,:]
u_train = torch.from_numpy(u_star[idx,:]).float()
X = torch.from_numpy(X_train[:,0:1]).requires_grad_(True).float()     
T = torch.from_numpy(X_train[:,1:2]).requires_grad_(True).float()

In [7]:
class Data(Dataset):

  def __init__(self, X, T, u_train):
    self.x = torch.cat([X, T], axis=1).float()
    self.y = u_train
    self.len = u_train.shape[0]
  
  def __getitem__(self, index):    
      return self.x[index], self.y[index]
  
  def __len__(self):
      return self.len

In [8]:
data_set = Data(X, T, u_train)
train_loader = DataLoader(dataset=data_set, batch_size=128)

# ANN

In [9]:
class ANN(nn.Module):

  def __init__(self, layers):
    super(ANN, self).__init__()
    self.layers = nn.ModuleList()
    for i, j in zip(layers, layers[1:]):
      linear = nn.Linear(i, j)
      nn.init.xavier_normal_(linear.weight.data, gain = 1.0)
      nn.init.zeros_(linear.bias.data)
      self.layers.append(linear)
  
  def forward(self, x):
    L = len(self.layers)
    for l, transform in enumerate(self.layers):
      if l < L-1:
        x = torch.tanh(transform(x))
      else:
        x = transform(x)
    return x   

In [10]:
ann_model = ANN(layers)
optimizer = torch.optim.Adam(ann_model.parameters(), lr = 0.0005)

In [11]:
def ANN_train(model, train_loader, optimizer, X, T):
  epochs = 5500
  mse = nn.MSELoss()
  for epoch in range(epochs):
    for x1, y1 in train_loader:
      model.train()
      optimizer.zero_grad()
      yhat = model(x1)
      loss1 = mse(yhat, y1)
      loss = loss1
      loss.backward()
      optimizer.step()
    if epoch % 100 == 0:
      print('Epoch:', epoch, 'Loss: %.5e' % (loss.item()))
  return

In [12]:
ANN_train(ann_model, train_loader, optimizer, X, T)

Epoch: 0 Loss: 2.28643e-01
Epoch: 100 Loss: 4.80102e-03
Epoch: 200 Loss: 6.42995e-04
Epoch: 300 Loss: 3.10664e-04
Epoch: 400 Loss: 2.18946e-04
Epoch: 500 Loss: 3.57881e-04
Epoch: 600 Loss: 1.18056e-04
Epoch: 700 Loss: 1.15186e-04
Epoch: 800 Loss: 1.07899e-04
Epoch: 900 Loss: 9.47318e-05
Epoch: 1000 Loss: 5.21400e-05
Epoch: 1100 Loss: 4.69550e-05
Epoch: 1200 Loss: 4.38040e-05
Epoch: 1300 Loss: 5.18826e-05
Epoch: 1400 Loss: 1.80649e-05
Epoch: 1500 Loss: 2.28271e-05
Epoch: 1600 Loss: 5.16983e-05
Epoch: 1700 Loss: 6.23666e-05
Epoch: 1800 Loss: 2.69692e-05
Epoch: 1900 Loss: 4.20702e-05
Epoch: 2000 Loss: 2.97366e-05
Epoch: 2100 Loss: 3.83605e-05
Epoch: 2200 Loss: 7.83006e-06
Epoch: 2300 Loss: 3.47090e-05
Epoch: 2400 Loss: 3.27041e-05
Epoch: 2500 Loss: 3.05767e-05
Epoch: 2600 Loss: 5.87759e-06
Epoch: 2700 Loss: 1.14113e-05
Epoch: 2800 Loss: 6.74329e-06
Epoch: 2900 Loss: 2.85318e-06
Epoch: 3000 Loss: 4.19800e-05
Epoch: 3100 Loss: 3.99287e-06
Epoch: 3200 Loss: 3.34585e-06
Epoch: 3300 Loss: 3.38

In [13]:
ann_model.eval()

ANN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=25, bias=True)
    (1-7): 7 x Linear(in_features=25, out_features=25, bias=True)
    (8): Linear(in_features=25, out_features=1, bias=True)
  )
)

In [14]:
X_ = torch.from_numpy(X_star[:,0:1]).requires_grad_(True).float()     
T_ = torch.from_numpy(X_star[:,1:2]).requires_grad_(True).float()
xf = torch.cat([X_, T_], axis=1)
uf = ann_model(xf)
u_x = grad(uf.sum(), X_, retain_graph = True, create_graph = True)[0]
u_xx = grad(u_x.sum(), X_, retain_graph = True, create_graph = True)[0]
u_t = grad(uf.sum(), T_, retain_graph = True, create_graph = True)[0]

In [30]:
ind = abs(uf.detach().numpy()-u_star).reshape(-1).argsort()[:150]

In [31]:
A = np.hstack([(-uf*u_x).detach().numpy(), u_xx.detach().numpy()])
B = u_t.detach().numpy()

l_1, l_2 = np.linalg.lstsq(A[ind], B[ind], rcond=None)[0]

In [32]:
print('Lambda1 Pred:', round(l_1[0],8), '  ; Lambda1 Actual:', 1.0)
print('Lambda2 Pred:', round(l_2[0],8), '  ; Lambda2 Actual:', round(0.01/np.pi,8))

Lambda1 Pred: 0.7976258   ; Lambda1 Actual: 1.0
Lambda2 Pred: 0.00069169   ; Lambda2 Actual: 0.0031831
